### Preparing the data

In [68]:
import pandas as pd
df = pd.read_csv("datasets/police_data/police.csv")
df.head()

,state,stop_date,stop_time,county_name,driver_gender,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district
0,RI,2005-01-04,12:55,NaN,M,White,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False,Zone X4
1,RI,2005-01-23,23:15,NaN,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone K3
2,RI,2005-02-17,04:15,NaN,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X4
3,RI,2005-02-20,17:15,NaN,M,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False,Zone X1
4,RI,2005-02-24,01:20,NaN,F,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X3


In [69]:
df.info()
# county_name all null, search_type mostly null (probably due to search_conducted=False), 
# many columns have missing values, # stop_date and stop_time should be converted to datetime, 
# violation and violation_raw seem to have similar values, # is_arrested should be a bool, 
# stop_duration would provide more info as a numerical variable

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   state               91741 non-null  object 
 1   stop_date           91741 non-null  object 
 2   stop_time           91741 non-null  object 
 3   county_name         0 non-null      float64
 4   driver_gender       86536 non-null  object 
 5   driver_race         86539 non-null  object 
 6   violation_raw       86539 non-null  object 
 7   violation           86539 non-null  object 
 8   search_conducted    91741 non-null  bool   
 9   search_type         3307 non-null   object 
 10  stop_outcome        86539 non-null  object 
 11  is_arrested         86539 non-null  object 
 12  stop_duration       86539 non-null  object 
 13  drugs_related_stop  91741 non-null  bool   
 14  district            91741 non-null  object 
dtypes: bool(2), float64(1), object(12)
memory usage: 9.3+

In [70]:
df.shape

(91741, 15)

In [71]:
df.isnull().sum()

state                     0
stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5205
driver_race            5202
violation_raw          5202
violation              5202
search_conducted          0
search_type           88434
stop_outcome           5202
is_arrested            5202
stop_duration          5202
drugs_related_stop        0
district                  0
dtype: int64

In [72]:
df["state"].value_counts()
# all values RI, no predictive value

RI    91741
Name: state, dtype: int64

In [73]:
df["driver_gender"].value_counts()

M    62762
F    23774
Name: driver_gender, dtype: int64

In [74]:
df["driver_race"].value_counts()

White       61872
Black       12285
Hispanic     9727
Asian        2390
Other         265
Name: driver_race, dtype: int64

In [75]:
df[["violation", "violation_raw"]].value_counts()
# violation_raw contains more detail

violation            violation_raw                   
Speeding             Speeding                            48424
Moving violation     Other Traffic Violation             16224
Equipment            Equipment/Inspection Violation      10922
Registration/plates  Registration Violation               3703
Seat belt            Seatbelt Violation                   2856
Other                Special Detail/Directed Patrol       2467
                     Call for Service                     1392
                     Motorist Assist/Courtesy              205
                     Violation of City/Town Ordinance      181
                     APB                                    91
                     Suspicious Person                      56
                     Warrant                                18
dtype: int64

In [76]:
df["search_conducted"].value_counts()

False    88434
True      3307
Name: search_conducted, dtype: int64

In [77]:
df["search_type"].value_counts()
# multiple values, could turn each into a bool

Incident to Arrest                                          1290
Probable Cause                                               924
Inventory                                                    219
Reasonable Suspicion                                         214
Protective Frisk                                             164
Incident to Arrest,Inventory                                 123
Incident to Arrest,Probable Cause                            100
Probable Cause,Reasonable Suspicion                           54
Incident to Arrest,Inventory,Probable Cause                   35
Probable Cause,Protective Frisk                               35
Incident to Arrest,Protective Frisk                           33
Inventory,Probable Cause                                      25
Protective Frisk,Reasonable Suspicion                         19
Incident to Arrest,Inventory,Protective Frisk                 18
Incident to Arrest,Probable Cause,Protective Frisk            13
Inventory,Protective Fris

In [78]:
df[["stop_outcome", "is_arrested"]].value_counts()

stop_outcome      is_arrested
Citation          False          77092
Warning           False           5137
Arrest Driver     True            2735
No Action         False            625
N/D               False            607
Arrest Passenger  True             343
dtype: int64

In [79]:
df["stop_duration"].value_counts()

0-15 Min     69579
16-30 Min    13740
30+ Min       3220
Name: stop_duration, dtype: int64

In [80]:
df["drugs_related_stop"].value_counts()

False    90879
True       862
Name: drugs_related_stop, dtype: int64

In [81]:
df["district"].value_counts()

Zone X4    24279
Zone K3    20405
Zone K2    18397
Zone X3    17013
Zone K1     8678
Zone X1     2969
Name: district, dtype: int64

In [82]:
df.drop(columns=["state", "county_name"], inplace=True)

In [83]:
df.dropna(subset=["driver_gender"], inplace=True)

In [84]:
df["is_arrested"] = df["is_arrested"].astype(bool)

In [85]:
df["stop_date"] = df["stop_date"].str.replace("/", "-")
combined = df["stop_date"].str.cat(df["stop_time"], sep=" ")
df["stop_datetime"] = pd.to_datetime(combined)
df.drop(columns=["stop_date", "stop_time"], inplace=True)
df.head()

,driver_gender,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,district,stop_datetime
0,M,White,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False,Zone X4,2005-01-04 12:55:00
1,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone K3,2005-01-23 23:15:00
2,M,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X4,2005-02-17 04:15:00
3,M,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False,Zone X1,2005-02-20 17:15:00
4,F,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,Zone X3,2005-02-24 01:20:00


In [86]:
df.set_index("stop_datetime", inplace=True)

In [87]:
df.isnull().sum()

driver_gender             0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           83229
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
district                  0
dtype: int64